In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



## TaxiStat
In this project I analyze a database containing over 170,000,000 taxi trips <br />
in NYC over the year 2013.

These are some dataset facts:

-Provided by the TLC of NYC.

-About 18 GB of data

-Data was imported into a MySQL database and is processed using Pandas and Numpy.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import mysql.connector
from bokeh.plotting import figure, output_file, show
from bokeh.plotting import *
from bokeh.models import Range1d
from bokeh.embed import components
from datetime import date, timedelta, datetime, time
from bokeh.embed import notebook_div
from bokeh.io import output_notebook
from identify_points import get_center_id, get_center_coordinates
import seaborn as sns
import networkx as nx
output_notebook()
import itertools
from cartesian import cartesian
import pickle

try:
    connect = mysql.connector.connect(host="localhost", user="hernan",  passwd="hernan", db="trips")
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)

In [ ]:
def get_time_slot(t):
    time_slot = 2*(t.hour) + (t.minute)/30
    return time_slot

In [ ]:
t1 = '2013-01-01 00:00:00'
t2 = '2013-01-01 23:59:59'
t1_datetime = pd.to_datetime(t1)
t2_datetime = pd.to_datetime(t2)

In [ ]:
t1_datetime.time()

In [ ]:
sql_query = "SELECT id, fare_amount, trip_distance, trip_time_in_secs, pickup_datetime, pickup_longitude, pickup_latitude FROM taxi_trip WHERE pickup_datetime BETWEEN '"+t1+"' AND '"+t2+"'"

In [ ]:
df_mysql = pd.read_sql(sql_query, con=connect)

To give a sense of what the data looks like in the Pandas<br />
here I show the first line of the DataFrame.<br />
(Note that there is a lot more data in the MySQL database<br />
that I'm not importing into the DataFrame for now.

Let's look at taxi patterns for January 1, 2013 ...

In [ ]:
time_resolution_in_minutes = 30
offset = pd.DateOffset(minutes=time_resolution_in_minutes)

In [ ]:
time_bins = pd.date_range(start=t1_datetime,end=t2_datetime, freq = offset)

In [ ]:
(time_bins[7].minute)/30

In [ ]:
df_binned = pd.DataFrame(index=time_bins[:-1], columns=['number_of_trips','fare_amount','trip_distance','trip_time_in_secs'])

In [ ]:
for time_index in range(len(time_bins)-1):
    data_bin = df_mysql[(df_mysql['pickup_datetime']>=time_bins[time_index]) & (df_mysql['pickup_datetime']<time_bins[time_index+1])]
    df_binned.ix[time_index]['number_of_trips']=len(data_bin)
    df_binned.ix[time_index]['fare_amount']=data_bin['fare_amount'].mean()
    df_binned.ix[time_index]['trip_distance']=data_bin['trip_distance'].mean()
    df_binned.ix[time_index]['trip_time_in_secs']=data_bin['trip_time_in_secs'].mean()

Then bin the results by time of pick up into 30-minute bins and look <br />
at the mean fare amount, trip distance, trip time, and number of trips.

In [ ]:
from math import pi
TOOLS="pan,wheel_zoom,box_zoom,reset,save"
p1 = figure(tools=TOOLS, plot_width=500, plot_height=500, x_axis_type='datetime', x_axis_label='Date and Time')
p1.line(df_binned.index,df_binned['number_of_trips']/1000,line_width=2, color="blue", legend="number of trips/1000")
p1.line(df_binned.index,df_binned['fare_amount'],line_width=2, color="red", legend="fare amount")
p1.line(df_binned.index,df_binned['fare_amount'].mean(),line_width=2, color="red", legend="Mean fare amount")
p1.line(df_binned.index,df_binned['trip_time_in_secs']/100,line_width=2, color="green", legend="Trip time in secs/100")
p1.line(df_binned.index,df_binned['trip_time_in_secs'].mean()/100,line_width=2, color="green", legend="Mean trip time in secs/100")
p1.line(df_binned.index,df_binned['trip_distance'],line_width=2, color="yellow", legend="Trip distance")
p1.line(df_binned.index,df_binned['trip_distance'].mean(),line_width=2, color="yellow", legend="Mean trip distance")
p1.xaxis.major_label_orientation = pi/2
p1.border_fill = 'whitesmoke'
p1.background_fill = 'beige'
p1.toolbar_location=None

In [ ]:
show(p1)

Now let's look at more microscopic data. Within the same timeframe, <br />
let's see the same timeseries for an area of 3 by 3 blocks around Chelsea,<br />
and also another 3 by 3 area around midtown.

In [ ]:
df_binned_chelsea = pd.DataFrame(index=time_bins[:-1], columns=['number_of_trips','fare_amount','trip_distance','trip_time_in_secs'])
df_binned_midtown = pd.DataFrame(index=time_bins[:-1], columns=['number_of_trips','fare_amount','trip_distance','trip_time_in_secs'])

In [ ]:
for time_index in range(len(time_bins)-1):
    data_bin_chelsea = df_mysql[(df_mysql['pickup_datetime']>=time_bins[time_index]) & \
                                (df_mysql['pickup_datetime']<time_bins[time_index+1]) & \
                                (df_mysql['pickup_longitude']>=-73.977637) & \
                                (df_mysql['pickup_longitude']<-73.970969) & \
                                (df_mysql['pickup_latitude']>=40.757924) & \
                                (df_mysql['pickup_latitude']<40.758359)]
    
    data_bin_midtown = df_mysql[(df_mysql['pickup_datetime']>=time_bins[time_index]) & \
                                (df_mysql['pickup_datetime']<time_bins[time_index+1]) & \
                                (df_mysql['pickup_longitude']>=-74.007291) & \
                                (df_mysql['pickup_longitude']<-73.996965) & \
                                (df_mysql['pickup_latitude']>=40.742209) & \
                                (df_mysql['pickup_latitude']<40.743481)]
    
    df_binned_chelsea.ix[time_index]['number_of_trips']=len(data_bin_chelsea)
    df_binned_chelsea.ix[time_index]['fare_amount']=data_bin_chelsea['fare_amount'].mean()
    df_binned_chelsea.ix[time_index]['trip_distance']=data_bin_chelsea['trip_distance'].mean()
    df_binned_chelsea.ix[time_index]['trip_time_in_secs']=data_bin_chelsea['trip_time_in_secs'].mean()
    
    df_binned_midtown.ix[time_index]['number_of_trips']=len(data_bin_midtown)
    df_binned_midtown.ix[time_index]['fare_amount']=data_bin_midtown['fare_amount'].mean()
    df_binned_midtown.ix[time_index]['trip_distance']=data_bin_midtown['trip_distance'].mean()
    df_binned_midtown.ix[time_index]['trip_time_in_secs']=data_bin_midtown['trip_time_in_secs'].mean()

In [ ]:
df_binned_chelsea = df_binned_chelsea.fillna(0)
df_binned_midtown = df_binned_midtown.fillna(0)

In [ ]:
TOOLS="pan,wheel_zoom,box_zoom,reset,save"
p2 = figure(tools=TOOLS, plot_width=400, plot_height=400, x_axis_type='datetime', x_axis_label='Date and Time',title="Chelsea")
p2.line(df_binned_chelsea.index,df_binned_chelsea['number_of_trips'],line_width=2, color="blue", legend="number of trips")
p2.line(df_binned_chelsea.index,df_binned_chelsea['fare_amount'],line_width=2, color="red", legend="fare amount")
p2.line(df_binned_chelsea.index,df_binned_chelsea['fare_amount'].mean(),line_width=2, color="red", legend="Mean fare amount")
p2.line(df_binned_chelsea.index,df_binned_chelsea['trip_time_in_secs']/10,line_width=2, color="green", legend="Trip time in secs/100")
p2.line(df_binned_chelsea.index,df_binned_chelsea['trip_time_in_secs'].mean()/10,line_width=2, color="green", legend="Mean trip time in secs/100")
p2.line(df_binned_chelsea.index,df_binned_chelsea['trip_distance'],line_width=2, color="yellow", legend="Trip distance")
p2.line(df_binned_chelsea.index,df_binned_chelsea['trip_distance'].mean(),line_width=2, color="yellow", legend="Mean trip distance")
p2.xaxis.major_label_orientation = pi/2
p2.background_fill = 'beige'
p2.border_fill = 'whitesmoke'


In [ ]:
TOOLS="pan,wheel_zoom,box_zoom,reset,save"
p3 = figure(tools=TOOLS, plot_width=400, plot_height=400, x_axis_type='datetime', x_axis_label='Date and Time',title="Midtown")
p3.line(df_binned_midtown.index,df_binned_midtown['number_of_trips'],line_width=2, color="blue")
p3.line(df_binned_midtown.index,df_binned_midtown['fare_amount'],line_width=2, color="red")
p3.line(df_binned_midtown.index,df_binned_midtown['fare_amount'].mean(),line_width=2, color="red")
p3.line(df_binned_midtown.index,df_binned_midtown['trip_time_in_secs']/10,line_width=2, color="green")
p3.line(df_binned_midtown.index,df_binned_midtown['trip_time_in_secs'].mean()/10,line_width=2, color="green")
p3.line(df_binned_midtown.index,df_binned_midtown['trip_distance'],line_width=2, color="yellow")
p3.line(df_binned_midtown.index,df_binned_midtown['trip_distance'].mean(),line_width=2, color="yellow")
p3.xaxis.major_label_orientation = pi/2
p3.background_fill = 'beige'
p3.border_fill = 'whitesmoke'


In [ ]:
p = gridplot([[p3, p2]])
p.toolbar_location=None
p.border_fill = None

show(p)

# Handling location and time

In [ ]:
my_latitude = 40.748347 
my_longitude = -73.999425
my_datetime = pd.to_datetime('2015-09-15 15:00:00')
time_resolution_in_minutes = 30
my_day_of_week = my_datetime.dayofweek
my_time = my_datetime.time()
time_delta = timedelta(minutes=time_resolution_in_minutes)
my_time_plus_delta = (my_datetime + time_delta).time()
offset = pd.DateOffset(minutes=time_resolution_in_minutes)

In [ ]:
t1 = '2013-01-01 00:00:00'
t2 = '2013-01-01 23:59:59'
t1_datetime = pd.to_datetime(t1)
t2_datetime = pd.to_datetime(t2)
time_bins = pd.date_range(start=t1_datetime,end=t2_datetime, freq = offset)

In [ ]:
my_center_id = get_center_id(my_longitude,my_latitude,x_min=x_min,x_max=x_max,y_min=y_min,y_max=y_max,mesh_space=mesh_space)

# Network

Let's make a network.
For a given mesh grid, create directed weighted networks every 30 mins where 
nodes are centers and a link appears if there is a trip between that center
and another one. THe weight is the number of trips within that half hour.

In [ ]:
x_min = -74.293396 #longitude  SW: 40.481965, -74.293396 NE:40.911486, -73.733866
x_max = -73.733866 #longitude
y_min = 40.481965 #latitude
y_max = 40.911486 #latitude
mesh_space = 0.01

centers_x,centers_rx=np.linspace(x_min,x_max,(x_max-x_min)/mesh_space,retstep="True")
centers_y,centers_ry=np.linspace(y_min,y_max,(y_max-y_min)/mesh_space,retstep="True")

In [ ]:
z=cartesian([centers_x,centers_y])

In [ ]:
centers_df = pd.DataFrame(z,columns=['longitude','latitude'])

In [ ]:
centers_df['center_id'] = centers_df.apply(lambda x: get_center_id(x['longitude'], x['latitude'],x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max,mesh_space=mesh_space), axis=1)

In [ ]:
centers_df.to_pickle('centers_long_lat_id_mesh_space='+str(mesh_space)+'.pickle')

In [ ]:
for day_of_week in range(2,7):
    #print 'day_of_week', day_of_week
    
    sql_query_full_day = 'SELECT id, fare_amount, trip_distance, \
    trip_time_in_secs, pickup_longitude, pickup_latitude,\
    dropoff_longitude, dropoff_latitude, pickup_datetime \
    FROM taxi_trip WHERE WEEKDAY(pickup_datetime) = '+str(day_of_week) +\
    ' AND pickup_longitude >=' +str(x_min)+ ' AND pickup_longitude <' +str(x_max)+\
    ' AND pickup_latitude >=' +str(y_min)+ ' AND pickup_latitude<' +str(y_max)
    
    df_mysql2 = pd.read_sql(sql_query_full_day, con=connect)
    df_mysql2['pickup_center']=df_mysql2.apply(lambda x: get_center_id(x['pickup_longitude'], x['pickup_latitude'],x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max,mesh_space=mesh_space), axis=1)
    df_mysql2['dropoff_center']=df_mysql2.apply(lambda x: get_center_id(x['dropoff_longitude'], x['dropoff_latitude'],x_min=x_min, x_max=x_max, y_min=y_min, y_max=y_max,mesh_space=mesh_space), axis=1)
        
    df_mysql2['pickup_time_slot'] = df_mysql2.apply(lambda x: get_time_slot(x['pickup_datetime']),axis=1)
    #df_mysql2['dropoff_time_slot'] = df_mysql2.apply(lambda x: get_time_slot(x['dropoff_datetime']),axis=1)

    #df_mysql2 = df_mysql2[['id','fare_amount','trip_distance','trip_time_in_secs','pickup_center','dropoff_center','pickup_time_slot']]

    grouped = df_mysql2[['fare_amount','trip_distance','trip_time_in_secs','pickup_time_slot']].groupby('pickup_time_slot', as_index=True)
    grouped = grouped.aggregate(np.mean)
    #grouped = grouped.set_index('pickup_time_slot')
    grouped['number_of_trips_from_pickup_center'] = (df_mysql2.groupby('pickup_time_slot')).size()
    
    network = df_mysql2.groupby(['pickup_time_slot','pickup_center','dropoff_center'],sort=True).size()    
    
    #data_dict = {"data" : grouped, "networks": {}}
    #for tt in range(48):
    #    G = nx.MultiDiGraph()
    #    nn = df_mysql2[df_mysql2['pickup_time_slot'] == tt][['pickup_center','dropoff_center']]
    #    G.add_edges_from(np.array(nn))
    #    data_dict["networks"][tt] = G
    
    output = open('main_data_mesh_space='+str(mesh_space)+'_day='+str(day_of_week)+'.pickle','wb')
    pickle.dump(grouped,output)
    output.close()

    output = open('network_mesh_space='+str(mesh_space)+'_day='+str(day_of_week)+'.pickle','wb')
    pickle.dump(network,output)
    output.close()
